In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from datetime import datetime
import sqlite3

import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [31]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

BASE_URL = "https://www.camara.leg.br/noticias/ultimas"
print("✅ Scraper da Câmara pronto!")


✅ Scraper da Câmara pronto!


In [32]:
DATABASE_NAME = "internet_governance_news.db"

def create_database():
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            date TEXT,
            author TEXT,
            url TEXT UNIQUE,
            source TEXT
        )
    """)
    conn.commit()
    conn.close()
    print("✅ Banco e tabela 'articles' prontos!")

create_database()


✅ Banco e tabela 'articles' prontos!


In [33]:
def insert_article(title, date, author, url, source):
    conn = sqlite3.connect(DATABASE_NAME)
    cursor = conn.cursor()
    try:
        cursor.execute("""
            INSERT INTO articles (title, date, author, url, source)
            VALUES (?, ?, ?, ?, ?)
        """, (title, date, author, url, source))
        conn.commit()
        return True
    except sqlite3.IntegrityError:
        return False
    finally:
        conn.close()


In [34]:
# %%
def load_articles_from_db():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("""
        SELECT *
        FROM articles
        ORDER BY date DESC
    """, conn)
    conn.close()
    return df

df_db = load_articles_from_db()
display(df_db.head())
print(f"📦 Total no banco: {len(df_db)} registros")


,id,title,date,author,url,source
0,74,"Em duas décadas, proporção de lares urbanos br...",31 OUT 2024,CGI.br Notícias,https://cgi.br/noticia/releases/em-duas-decada...,CGI Notícias
1,188,"No FIB13, especialistas apontam expectativas e...",31 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/no-fib-13-espe...,CGI Notícias
2,12,Mais acessível e completo: conheça o novo site...,30 SET 2025,CGI.br Notícias,https://cgi.br/noticia/notas/mais-acessivel-e-...,CGI Notícias
3,13,Evento em São Paulo celebra 15 anos de SACI-Ad...,30 SET 2025,CGI.br Notícias,https://cgi.br/noticia/releases/evento-em-sao-...,CGI Notícias
4,189,Abertas as inscrições para o 13º Fórum da Inte...,30 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/abertas-as-ins...,CGI Notícias


📦 Total no banco: 609 registros


In [35]:
def montar_url(pagina):
    if pagina == 1:
        return BASE_URL
    return f"{BASE_URL}?pagina={pagina}"


def extrair_paragrafos(url):
    try:
        r = requests.get(url, headers=HEADERS, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        ps = soup.find_all("p")
        textos = [
            p.get_text(strip=True)
            for p in ps
            if len(p.get_text(strip=True).split()) > 10
        ]
        return textos[:5] if textos else ["NA"]
    except:
        return ["NA"]


In [36]:
noticias = []
TOTAL_PAGES = 5

for pagina in range(TOTAL_PAGES, 0, -1):
    url = montar_url(pagina)
    print(f"📄 Coletando página {pagina}: {url}")

    r = requests.get(url, headers=HEADERS, timeout=10)
    if r.status_code != 200:
        print("⚠️ Erro ao acessar página")
        continue

    soup = BeautifulSoup(r.text, "html.parser")
    artigos = soup.find_all("article", class_="g-chamada")
    print(f"   {len(artigos)} notícias encontradas")

    for art in artigos:
        titulo_tag = art.find("h3", class_="g-chamada__titulo")
        data_tag = art.find("span", class_="g-chamada__data")

        if not titulo_tag:
            continue

        a = titulo_tag.find("a")
        titulo = a.get_text(strip=True)
        link = a["href"]
        data_raw = data_tag.get_text(strip=True) if data_tag else "NA"

        paragrafos = extrair_paragrafos(link)

        noticias.append({
            "titulo": titulo,
            "data": data_raw,
            "link": link,
            "paragrafos": " || ".join(paragrafos),
            "fonte": "Câmara dos Deputados"
        })

        insert_article(
            title=titulo,
            date=data_raw,
            author="Agência Câmara",
            url=link,
            source="Câmara dos Deputados"
        )

    time.sleep(1)

print(f"\n✅ Total coletado: {len(noticias)} notícias")

df_camara = pd.DataFrame(noticias)
display(df_camara.head())


📄 Coletando página 5: https://www.camara.leg.br/noticias/ultimas?pagina=5
   10 notícias encontradas
📄 Coletando página 4: https://www.camara.leg.br/noticias/ultimas?pagina=4
   10 notícias encontradas
📄 Coletando página 3: https://www.camara.leg.br/noticias/ultimas?pagina=3
   10 notícias encontradas
📄 Coletando página 2: https://www.camara.leg.br/noticias/ultimas?pagina=2
   10 notícias encontradas
📄 Coletando página 1: https://www.camara.leg.br/noticias/ultimas
   10 notícias encontradas

✅ Total coletado: 50 notícias


,titulo,data,link,paragrafos,fonte
0,Comissão aprova projeto que amplia isenção de ...,15/12/2025 17:06,https://www.camara.leg.br/noticias/1232872-com...,Marina Ramos / Câmara dos DeputadosDeputado Pe...,Câmara dos Deputados
1,Comissão debate falta de energia elétrica nos ...,15/12/2025 17:00,https://www.camara.leg.br/noticias/1232431-com...,Pablo Valadares/Câmara dos DeputadosYury do Pa...,Câmara dos Deputados
2,Comissão de Constituição e Justiça aprova equi...,15/12/2025 16:08,https://www.camara.leg.br/noticias/1232889-com...,"A proposta pode seguir para análise do Senado,...",Câmara dos Deputados
3,Comissão aprova regras para uso de drones arma...,15/12/2025 15:55,https://www.camara.leg.br/noticias/1232704-com...,O projeto de lei segue em análise na Câmara do...,Câmara dos Deputados
4,Comissão aprova objetivos para ações no Dia Na...,15/12/2025 15:39,https://www.camara.leg.br/noticias/1232773-com...,"Proposta segue para o Senado, caso não haja re...",Câmara dos Deputados


In [41]:
def load_articles():
    conn = sqlite3.connect(DATABASE_NAME)
    df = pd.read_sql("""
        SELECT * FROM articles
        ORDER BY date DESC
    """, conn)
    conn.close()
    return df

df_db = load_articles()
print(f"📦 Total no banco: {len(df_db)} registros")
display(df_db.head(20))


📦 Total no banco: 609 registros


,id,title,date,author,url,source
0,74,"Em duas décadas, proporção de lares urbanos br...",31 OUT 2024,CGI.br Notícias,https://cgi.br/noticia/releases/em-duas-decada...,CGI Notícias
1,188,"No FIB13, especialistas apontam expectativas e...",31 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/no-fib-13-espe...,CGI Notícias
2,12,Mais acessível e completo: conheça o novo site...,30 SET 2025,CGI.br Notícias,https://cgi.br/noticia/notas/mais-acessivel-e-...,CGI Notícias
3,13,Evento em São Paulo celebra 15 anos de SACI-Ad...,30 SET 2025,CGI.br Notícias,https://cgi.br/noticia/releases/evento-em-sao-...,CGI Notícias
4,189,Abertas as inscrições para o 13º Fórum da Inte...,30 MAI 2023,CGI.br Notícias,https://cgi.br/noticia/releases/abertas-as-ins...,CGI Notícias
5,119,Declaração final do NETmundial+10 apresenta re...,30 ABR 2024,CGI.br Notícias,https://cgi.br/noticia/releases/declaracao-fin...,CGI Notícias
6,83,Nota de falecimento: Mariana Stanton,29 SET 2024,CGI.br Notícias,https://cgi.br/noticia/notas/nota-de-falecimen...,CGI Notícias
7,159,Nota de falecimento: Luiz Gonzaga Lauschner,29 SET 2023,CGI.br Notícias,https://cgi.br/noticia/notas/nota-de-falecimen...,CGI Notícias
8,38,Fórum da Internet no Brasil resgata legado da ...,29 MAI 2025,CGI.br Notícias,https://cgi.br/noticia/releases/forum-da-inter...,CGI Notícias
9,39,Prêmio Destaques homenageia 12 personalidades ...,29 MAI 2025,CGI.br Notícias,https://cgi.br/noticia/releases/premio-destaqu...,CGI Notícias


In [42]:
keywords = ['digital', 'internet', 'IA', 'tecnologia', 'dados', 'privacidade']
pattern = r'|'.join(keywords)

df_filt = df_camara[
    df_camara['titulo'].str.contains(pattern, case=False, na=False, regex=True) |
    df_camara['paragrafos'].str.contains(pattern, case=False, na=False, regex=True)
].copy()

print(f"{len(df_filt)} notícias filtradas (de {len(df_camara)})")
display(df_filt.head())


50 notícias filtradas (de 50)


,titulo,data,link,paragrafos,fonte
0,Comissão aprova projeto que amplia isenção de ...,15/12/2025 17:06,https://www.camara.leg.br/noticias/1232872-com...,Marina Ramos / Câmara dos DeputadosDeputado Pe...,Câmara dos Deputados
1,Comissão debate falta de energia elétrica nos ...,15/12/2025 17:00,https://www.camara.leg.br/noticias/1232431-com...,Pablo Valadares/Câmara dos DeputadosYury do Pa...,Câmara dos Deputados
2,Comissão de Constituição e Justiça aprova equi...,15/12/2025 16:08,https://www.camara.leg.br/noticias/1232889-com...,"A proposta pode seguir para análise do Senado,...",Câmara dos Deputados
3,Comissão aprova regras para uso de drones arma...,15/12/2025 15:55,https://www.camara.leg.br/noticias/1232704-com...,O projeto de lei segue em análise na Câmara do...,Câmara dos Deputados
4,Comissão aprova objetivos para ações no Dia Na...,15/12/2025 15:39,https://www.camara.leg.br/noticias/1232773-com...,"Proposta segue para o Senado, caso não haja re...",Câmara dos Deputados


In [43]:
def plot_charts(df):
    if df.empty:
        print("❌ Sem dados para gráficos")
        return

    # ------------------------------
    # Top 15
    # ------------------------------
    top15 = df.head(15).copy()
    top15['rank'] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x='rank',
        y='title',
        orientation='h',
        title='Top 15 Notícias – Internet Governance'
    )
    fig1.update_layout(height=600)
    fig1.show()

    # ------------------------------
    # Pizza por Fonte (BANCO)
    # ------------------------------
    # Fonte
    source_count = df["source"].value_counts().reset_index()
    source_count.columns = ["source", "count"]

    fig2 = px.pie(
        source_count,
        names="source",
        values="count",
        title="Distribuição por Fonte"
    )
    fig2.show()

    # ------------------------------
    # Nuvem de Palavras
    # ------------------------------
    text = ' '.join(df['title'].astype(str)).lower()
    words = re.findall(r'\b\w{4,}\b', text)

    wc = (
        pd.Series(words)
        .value_counts()
        .head(20)
        .reset_index()
    )
    wc.columns = ['palavra', 'freq']

    fig3 = px.treemap(
        wc,
        path=['palavra'],
        values='freq',
        title='Nuvem de Palavras – Títulos'
    )
    fig3.show()


In [44]:
plot_charts(df_db)